# License

Copyright 2018 Hamaad Musharaf Shah

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Using Bidirectional Generative Adversarial Networks to estimate Value-at-Risk for Market Risk Management
## Author: Hamaad Shah

---

We will explore the use of Bidirectional Generative Adversarial Networks (BiGAN) for market risk management: Estimation of portfolio risk measures such as Value-at-Risk (VaR). Generative Adversarial Networks (GAN) allow us to implicitly maximize the likelihood of complex distributions thereby allowing us to generate samples from such distributions - the key point here is the implicit maximum likelihood estimation principle whereby we do not specify what this complex distribution is parameterized as. Dealing with high dimensional data potentially coming from a complex distribution is a key aspect to market risk management among many other financial services use cases. GAN, specifically BiGAN for the purpose of this paper, will allow us to deal with potentially complex financial services data such that we do not have to explicitly specify a distribution such as a multidimensional Gaussian distribution.

## Market Risk Management: Value-at-Risk (VaR)

---

Let us begin by fetching a data set of 5 stocks from Yahoo. The stocks are Apple, Google, Microsoft, Intel and Box. We use a daily frequency for our data for the year 2016. We use the stock's daily closing prices to compute the continuously compounded returns: $\log\left(\frac{V_{t+1}}{V_{t}}\right) = \log(V_{t+1}) - \log(V_{t})$.

```r
# Set scientific notation penalty to high for better printing.
options(scipen = 999)

# Load required libraries.
library(ggplot2)
library(quantmod)
library(MASS)
library(scales)
library(reshape2)

# Let's take a year's worth of data.
start <- as.Date("2016-01-01")
end <- as.Date("2016-12-31")

# Let's get Apple, Google, Microsoft, Intel and Box data from Yahoo.
getSymbols(Symbols = "AAPL;GOOG;MSFT;INTC;BOX", 
           src = "yahoo", 
           from = start, 
           to = end)

# Create daily returns series based on close prices.
data <- data.frame("Apple" = as.numeric(AAPL[, "AAPL.Close"]),
                   "Google" = as.numeric(GOOG[, "GOOG.Close"]),
                   "Microsoft" = as.numeric(MSFT[, "MSFT.Close"]),
                   "Intel" = as.numeric(INTC[, "INTC.Close"]),
                   "Box" = as.numeric(BOX[, "BOX.Close"]))
ret.data <- data.frame(diff(x = as.matrix(log(x = data)), lag = 1))
colnames(ret.data) <- c("Apple", "Google", "Microsoft", "Intel", "Box")
```

Let's assume that we have an equal weight for each of the 5 assets in our portfolio. Based on this portfolio weights assumption we can calculate the portfolio returns.

```r
# Let's assume we have an equal weight for all assets in our 5 assets portfolio.
# Based on those weightings calculate the portfolio returns on a daily basis.
port.data <- data.frame(apply(X = ret.data, 
                              MARGIN = 1,
                              FUN = function(x) weighted.mean(x = x, 
                                                              w = rep(x = 1 / ncol(ret.data), ncol(ret.data)))))
colnames(port.data) <- "Portfolio.ret"

# Plot the daily portfolio returns data.
ggplot(data = port.data, 
       aes(x = Portfolio.ret)) +
geom_density(fill = "skyblue", alpha = 0.25) +
ggtitle("KDE plot of portfolio returns") +
xlab("Portfolio returns") +
ylab("Density") +
scale_x_continuous(labels = percent)
```

![](../R/portfolio_returns.png)

Let's estimate the expected returns vector, volatilities vector, correlation and variance-covariance matrices. The variance-covariance matrix is recovered from the estimated volatilities vector and correlation matrix: $\Omega = C \odot \sigma \sigma^{T}$ where $\odot$ is the Hadamard product, $C \in \mathbb{R}^{5 \times 5}$ and $\sigma \in \mathbb{R}^{5 \times 1}$. Portfolio volatility is estimated as: $w^{T}\Omega w$ where $w \in \mathbb{R}^{5 \times 1}$

```r
# Get mean returns vector.
est.ret.vector <- colMeans(x = ret.data)

# Get volatilities vector.
est.sd <- as.matrix(apply(X = ret.data, 
                          MARGIN = 2, 
                          FUN = sd))

# Get correlation matrix.
est.cor.matrix <- cor(x = ret.data)

# Use volatilies vector and correlation matrix to estimate variance-covariance matrix.
est.cov.matrix <- est.cor.matrix * (est.sd %*% t(est.sd))

# Given the earlier portfolio weights estimate portfolio volatility.
est.sd.port <- sqrt(t(as.matrix(rep(x = 1 / ncol(ret.data), ncol(ret.data)))) %*% 
                      est.cov.matrix %*% 
                      as.matrix(rep(x = 1 / ncol(ret.data), ncol(ret.data))))
```

We consider the 3 major methods used in market risk management, specifically for the estimation of VaR. Please note that there are multiple different methods for estimating VaR and other more coherent risk measures such as Conditional Value-at-Risk (CVaR) however we are only considering the few major ones.

## VaR: Variance-covariance method

---

The first one is the variance-covariance method and uses the estimated portfolio volatility $w^{T}\Omega w$ under the Gaussian assumption to estimate VaR. Let's assume we are attempting to estimate 1% VaR: This means that there is a 1% probability of obtaining a portfolio return of less than the VaR value. Using the variance-covariance approach the calculation is: $\left[\left(w^{T}\Omega w\right) \mathcal{N}^{-1}(1\%)\right] + w^{T}\mu$, where $\mu \in \mathbb{R}^{5 \times 1}$ is the expected returns vector.

```r
# Estimate VaR using variance-covariance method.
VaR.var.covar <- (est.sd.port * qnorm(p = 0.01, 
                                      mean = 0, 
                                      sd = 1)) + mean(port.data$Portfolio.ret)
```

## VaR: Historical simulation method
The second method is a non-parametric approach where we sample with replacement from the historical data to estimate a portfolio returns distribution. The 1% VaR is simply the appropriate quantile from this sampled portfolio returns distribution.

```r
# Estimate VaR using historical simulation method.
VaR.hist.sim <- quantile(x = sample(x = port.data$Portfolio.ret, 
                                    size = 1000, 
                                    replace = TRUE), 
                         probs = 0.01)
```

## VaR: Monte Carlo method

---

The third method is Monte Carlo sampling from a multidimensional Gaussian distribution using the aforementioned $mu$ and $\Omega$ parameters. Finally the 1% VaR is simply the appropriate quantile from this sampled portfolio returns distribution.

```r
# Estimate VaR using Monte Carlo simulation method.
# Simulate returns data from multidimensional Gaussian distribution.
sim.ret.data <- mvrnorm(n = 1000, 
                        mu = est.ret.vector, 
                        Sigma = est.cov.matrix)

# Get portfolio returns.
sim.port.data <- data.frame(apply(X = sim.ret.data, 
                                  MARGIN = 1,
                                  FUN = function(x) weighted.mean(x = x, 
                                                                  w = rep(x = 1 / ncol(sim.ret.data), ncol(sim.ret.data)))))
colnames(sim.port.data) <- "Portfolio.ret"

VaR.MC <- quantile(x = sim.port.data$Portfolio.ret, 
                   probs = 0.01)
```

## VaR: Estimates
The VaR estimates from the aforementioned 3 market risk management methods commonly used in banking are as follows:

```r
VaR <- data.frame("Var.Covar" = VaR.var.covar,
                  "Hist.Sim" = VaR.hist.sim,
                  "MC" = VaR.MC)
```

| VaR Method    | 1% VaR | 
| :------------- |-------------:|
| Variance-covariance | -2.87% | 
| Historical simulation | -3.65%  |
| Monte Carlo simulation | -2.63%  |

## Bidirectional Generative Adversarial Network (BiGAN)

---

The 2 main components to a Generative Adversarial Network (GAN) are the generator and the discriminator. These 2 components play an adversarial game against each other. In doing so the generator learns how to create realistic synthetic samples from noise, i.e., the latent space $z$, while the discriminator learns how to distinguish between a real sample and a synthetic sample. See the following article of mine for a detailed explanation of GAN: https://goo.gl/ZWYngw

BiGAN extends GAN by adding a third component: The encoder, which learns to map from data space $x$ to the latent space $z$. The objective of the generator remains the same while the objective of the discriminator is altered to classify between a real sample and a synthetic sample and additionally between a real encoding, i.e., given by the encoder, and a synthetic encoding, i.e., a sample from the latent space $z$.

### Generator

---

Assume that we have a prior belief on where the latent space $z$ lies: $p_{Z}(z)$. Given a draw from this latent space the generator $G$, a deep learner parameterized by $\theta_{G}$, outputs a synthetic sample.

$$
G(z|\theta_{G}): z \rightarrow x_{synthetic}
$$ 

### Encoder

---

This can be shown to be an inverse of the generator. Given a draw from the data space the encoder $E$, a deep learner parameterized by $\theta_{E}$, outputs a real encoding.

$$
E(x|\theta_{E}): x \rightarrow z
$$ 

### Discriminator

---

The discriminator $D$ is a deep learner parameterized by $\theta_{D}$ and it aims to classify if a sample is real or synthetic, i.e., if a sample is from the real data distribution,

$$
p_{X}(x)
$$ 

or the synthetic data distribution.

$$
p_{G}(x|z)
$$

Additionally it aims to classify whether an encoding is real,

$$
p_{E}(z|x)
$$

or synthetic.

$$
p_{Z}(z) 
$$

Let us denote the discriminator $D$ as follows.

$$
D(\{x, z\}|\theta_{D}): \{x, z\} \rightarrow [0, 1]
$$ 

We assume that the positive examples are real, i.e., $\{x, E(x|\theta_{E})\}$ while the negative examples are synthetic, i.e., $\{G(z|\theta_{G}), z\}$. 

### Game

---

The BiGAN has the following objective function, similar to the GAN.

$$
\min_{G(z|\theta_{G}), E(x|\theta_{E})} \max_{D(\{x, z\}|\theta_{D})} V(D(\{x, z\}|\theta_{D}), G(z|\theta_{G}), E(x|\theta_{E}))
$$

\begin{align*}
V(D(\{x, z\}|\theta_{D}), G(z|\theta_{G}), E(x|\theta_{E})) &= \mathbb{E}_{x \sim p_{X}(x)} \mathbb{E}_{z \sim p_{E}(z|x)} \log\left[{D(\{x, z\}|\theta_{D})}\right] + \mathbb{E}_{z \sim p_{Z}(z)} \mathbb{E}_{x \sim p_{G}(x|z)} \log\left[{1-D(\{x, z\}|\theta_{D})}\right] \\
&= \int_{x} p_{X}(x) \int_{z} p_{E}(z|x) \log\left[{D(\{x, z\}|\theta_{D})}\right] dz dx + \int_{z} p_{Z}(z) \int_{x} p_{G}(x|z) \log\left[{1 - D(\{x, z\}|\theta_{D})}\right] dx dz \\
&= \int_{\{x, z\}} p_{X}(x) p_{E}(z|x) \log\left[{D(\{x, z\}|\theta_{D})}\right] d\{x, z\} + \int_{\{x, z\}} p_{Z}(z) p_{G}(x|z) \log\left[{1 - D(\{x, z\}|\theta_{D})}\right] d\{x, z\} \\
&= \int_{\omega=\{x, z\}} \underbrace{p_{EX}(\omega) \log\left[{D(\omega|\theta_{D})}\right] + p_{GZ}(\omega) \log\left[{1 - D(\omega|\theta_{D})}\right]}_{J(D(\omega|\theta_{D}))} d\omega \\
\end{align*}

Let us take a closer look at the discriminator's objective function for a sample $\omega$.

\begin{align*}
J(D(\omega|\theta_{D})) &= p_{EX}(\omega) \log{D(\omega|\theta_{D})} + p_{GZ}(\omega) \log{(1 - D(\omega|\theta_{D}))} \\
\frac{\partial J(D(\omega|\theta_{D}))}{\partial D(\omega|\theta_{D})} &= \frac{p_{EX}(\omega)}{D(\omega|\theta_{D})} - \frac{p_{GZ}(\omega)}{(1 - D(\omega|\theta_{D}))} \\
0 &= \frac{p_{EX}(\omega)}{D^\ast(\omega|\theta_{D^\ast})} - \frac{p_{GZ}(\omega)}{(1 - D^\ast(\omega|\theta_{D^\ast}))} \\
p_{EX}(\omega)(1 - D^\ast(\omega|\theta_{D^\ast})) &= p_{GZ}(\omega)D^\ast(\omega|\theta_{D^\ast}) \\
p_{EX}(\omega) - p_{EX}(\omega)D^\ast(\omega|\theta_{D^\ast})) &= p_{GZ}(\omega)D^\ast(\omega|\theta_{D^\ast}) \\
p_{GZ}(\omega)D^\ast(\omega|\theta_{D^\ast}) + p_{EX}(\omega)D^\ast(\omega|\theta_{D^\ast})) &= p_{EX}(\omega) \\
D^\ast(\omega|\theta_{D^\ast}) &= \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)} 
\end{align*}

We have found the optimal discriminator given a generator and an encoder. Let us focus now on the generator's objective function and encoder's objective function; the goal of both of which is essentially to minimize the discriminator's objective function.

\begin{align*}
J(G(z|\theta_{G}), E(x|\theta_{E})) &= \mathbb{E}_{\omega \sim p_{EX}(\omega)} \log{D^\ast(\omega|\theta_{D^\ast})} + \mathbb{E}_{\omega \sim p_{GZ}(\omega)} \log{(1 - D^\ast(\omega|\theta_{D^\ast}))} \\
&= \mathbb{E}_{\omega \sim p_{EX}(\omega)} \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) + \mathbb{E}_{\omega \sim p_{GZ}(\omega)} \log{\bigg(1 - \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} \\
&= \mathbb{E}_{\omega \sim p_{EX}(\omega)} \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) + \mathbb{E}_{\omega \sim p_{GZ}(\omega)} \log{\bigg(\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} \\
&= \int_{\omega} p_{EX}(\omega) \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega
\end{align*}

We will note the Kullback–Leibler (KL) divergences in the above objective function for the generator and encoder.

$$
D_{KL}(P||Q) = \int_{x} p(x) \log\bigg(\frac{p(x)}{q(x)}\bigg) dx
$$

Recall the definition of a $\lambda$ divergence.

$$
D_{\lambda}(P||Q) = \lambda D_{KL}(P||\lambda P + (1 - \lambda) Q) + (1 - \lambda) D_{KL}(Q||\lambda P + (1 - \lambda) Q)
$$

If $\lambda$ takes the value of 0.5 this is then called the Jensen-Shannon (JS) divergence. This divergence is symmetric and non-negative.

$$
D_{JS}(P||Q) = 0.5 D_{KL}\bigg(P\bigg|\bigg|\frac{P + Q}{2}\bigg) + 0.5 D_{KL}\bigg(Q\bigg|\bigg|\frac{P + Q}{2}\bigg)
$$

Keeping this in mind let us take a look again at the objective function of the generator and the encoder.

\begin{align*}
J(G(z|\theta_{G}), E(x|\theta_{E})) &= \int_{\omega} p_{EX}(\omega) \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega \\
&= \int_{\omega} p_{EX}(\omega) \log{\bigg(\frac{2}{2}\frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{2}{2}\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega \\
&= \int_{\omega} p_{EX}(\omega) \log{\bigg(\frac{1}{2}\frac{1}{0.5}\frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{1}{2}\frac{1}{0.5}\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega \\
&= \int_{\omega} p_{EX}(\omega) \bigg[ \log(0.5) + \log{\bigg(\frac{p_{EX}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}} \bigg) \bigg] d\omega \\ &+ \int_{\omega} p_{GZ}(\omega) \bigg[\log(0.5) + \log{\bigg(\frac{p_{GZ}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}\bigg) \bigg] } d\omega \\
&= \log\bigg(\frac{1}{4}\bigg) + \int_{\omega} p_{EX}(\omega) \bigg[\log{\bigg(\frac{p_{EX}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}} \bigg) \bigg] d\omega \\ 
&+ \int_{\omega} p_{GZ}(\omega) \bigg[\log{\bigg(\frac{p_{GZ}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}\bigg) \bigg] } d\omega \\
&= -\log(4) + D_{KL}\bigg(p_{EX}(\omega)\bigg|\bigg|\frac{p_{EX}(\omega) + p_{GZ}(\omega)}{2}\bigg) + D_{KL}\bigg(p_{GZ}(\omega)\bigg|\bigg|\frac{p_{EX}(\omega) + p_{GZ}(\omega)}{2}\bigg) \\
&= -\log(4) + 2 \bigg(0.5 D_{KL}\bigg(p_{EX}(\omega)\bigg|\bigg|\frac{p_{EX}(\omega) + p_{GZ}(\omega)}{2}\bigg) + 0.5 D_{KL}\bigg(p_{GZ}(\omega)\bigg|\bigg|\frac{p_{EX}(\omega) + p_{GZ}(\omega)}{2}\bigg)\bigg) \\
&= -\log(4) + 2D_{JS}(p_{EX}(\omega)||p_{GZ}(\omega)) 
\end{align*}

It is clear from the objective function of the generator and encoder above that the global minimum value attained is $-\log(4)$ which occurs when the following holds.

$$
p_{EX}(\omega)=p_{GZ}(\omega)
$$

When the above holds the Jensen-Shannon divergence, i.e., $D_{JS}(p_{EX}(\omega)||p_{GZ}(\omega))$, will be zero. Hence we have shown that the optimal solution is as follows.

$$
p_{EX}(\omega)=p_{GZ}(\omega)
$$

Given the above result we can prove that the optimal discriminator will be $\frac{1}{2}$.

\begin{align*}
D^\ast(\omega|\theta_{D^\ast}) &= \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)} \\
 &= \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{EX}(\omega)} \\
 &= \frac{p_{EX}(\omega)}{2p_{EX}(\omega)} \\
 &= \frac{1}{2} \\
\end{align*}

## References

1. Goodfellow, I., Bengio, Y. and Courville A. (2016). Deep Learning (MIT Press).
2. Geron, A. (2017). Hands-On Machine Learning with Scikit-Learn & Tensorflow (O'Reilly).
3. Kingma, D. P., and Welling M. (2014). Auto-Encoding Variational Bayes (https://arxiv.org/abs/1312.6114).
4. http://scikit-learn.org/stable/#
5. https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
6. https://stackoverflow.com/questions/42177658/how-to-switch-backend-with-keras-from-tensorflow-to-theano
7. https://blog.keras.io/building-autoencoders-in-keras.html
8. https://keras.io
11. Chollet, F. (2018). Deep Learning with Python (Manning).
12. Hull, John C. (2010). Risk Management and Financial Institutions (Pearson).
13. https://towardsdatascience.com/automatic-feature-engineering-using-deep-learning-and-bayesian-inference-application-to-computer-7b2bb8dc7351
14. https://towardsdatascience.com/automatic-feature-engineering-using-generative-adversarial-networks-8e24b3c16bf3